**本代码的作用：**

    1、读取医学格式的T1AC原图，预处理；
        1、偏置场校正（可选）
        2、提取前景,然后剔除黑框
            1)自动阈值分割
            2)连通域分析,取出过小的连通域
            3)剔除纯背景的黑框
        3、用0和99.8百分位来归一化灰度值
        4、将3D图分为一系列的2D slice
    2、用模型前向推理获取肝脏、肿瘤预测掩膜
        先用2D liver模型前向推理每一张slice中的肝脏
        再用3D tumour模型前向推理肿瘤
    3、合成3D的mask
    4、后处理过滤虚假分割
    5、将mask生存nrrd文件，达到能在3D Slicer中和原图一起打开的效果。

In [1]:
import numpy as np
from glob import glob
from tqdm import tqdm
import os
import pandas as pd
import SimpleITK as sitk
from skimage import transform
from collections import Counter
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import skimage
from torchvision.utils import make_grid
import cv2
import torch
from utils import * 
import warnings

import sys
sys.path.append('../utils')
from util import *

os.environ['CUDA_VISIBLE_DEVICES'] = '6,7'

Missing colon in file PosixPath('/usr/local/lib/python3.8/dist-packages/matplotlib/mpl-data/matplotlibrc'), line 267 ('im')
Missing colon in file '/usr/local/lib/python3.8/dist-packages/matplotlib/mpl-data/matplotlibrc', line 267 ('im')


In [2]:
# get liver seg model
model_path = os.path.join("../../model/DeepLab_t1AC_liver", 'best_model.pth')
model_liver = get_model(model_path)
model_liver = model_liver.cuda()
# get lesion seg model
# model_path = os.path.join("../model/NestedUNet_t1AC_lesion", 'best_model.pth')
model_path = os.path.join("../../model/Nested_3DUNet_t1AC_lesion", 'best_model.pth')
model_lesion = get_model(model_path)
model_lesion = model_lesion.cuda()

In [3]:
source_dir = "../../data/for_auto_label/002-test"
save_path = "../../data/for_auto_label/002-test-auto-label"
# 读取AC.nrrd
# pat = os.path.join(source_dir,"*/AC","correct_AC.nii")
pat = os.path.join( r"../../data/V3/image_mask/a/*/AC","AC.nrrd" )
filepath_list = glob(pat)
newspacing = [1.0, 1.0, 1.0]
for filepath in tqdm(filepath_list):#[4:]
    # get sample_name
    sample_name = filepath.split('/')[-3]
    print(">>",sample_name)
    
    # read image
    image_sitk = sitk.ReadImage(filepath)
    oldsize = image_sitk.GetSize()
    
    # resample/rescale( by sitk )
    print("Size before resampling: ",oldsize)
    if newspacing is not None:
        image_sitk = resample_image3D(image_sitk,newspacing,method='Linear')
        print("Size after resampling: ",image_sitk.GetSize())
        
    # get array of image
    image_array = sitk.GetArrayFromImage(image_sitk).transpose((1,2,0))
    # get preprocessed image array
    image,(hmin, hmax, wmin, wmax, dmin, dmax),(lower,upper) = preprocess_for_inference(image_array)
    
    # get predicted liver mask
    print("getting mask_liver_3D")
    mask_liver_3D = get_predict_mask_for_3Dimage( 
        model_liver, 
        image,
        out_name='out'
    ).astype(np.uint8)
    
    image = image/2+0.5 # 回复给肿瘤分割模型用
    
    # post-process for predicted liver mask
    print("post-process for predicted liver mask")
    mask_liver_3D_post = get_post_process_liver_mask(mask_liver_3D)
    
    # get tumour mask
    print("getting mask_lesion_3D")
    model_mode = '3D'
    mask_lesion_3D,proba_lesion_3D = get_predict_lesion_mask_for_3Dimage(
        model_lesion,
        model_mode,
        image, 
        mask_liver_3D_post,
        out_name=None,
        pt=0.5)
    
    # get resized liver mask
    output_shape = (hmax-hmin+1, wmax-wmin+1, dmax-dmin+1)

    # recover bounding for mask_liver
    mask_liver = np.zeros_like(image_array)
    mask_liver[hmin:hmax+1, wmin:wmax+1, dmin:dmax+1] = mask_liver_3D_post

    # post-process lesion mask( (1)resize; (2)remove small connected domains )
    print("post process lesion mask")
    mask_lesion_3D_post = get_post_processed_lesion_mask(
        image,
        proba_lesion_3D,
        mask_liver_3D_post,
        output_shape=None,
        pt=0.5
    )
#     mask_lesion_3D_post = (proba_lesion_3D>0.5).astype(np.uint8)
    # recover bounding for mask_lesion
    mask_lesion = np.zeros_like(image_array)
    mask_lesion[hmin:hmax+1, wmin:wmax+1, dmin:dmax+1] = mask_lesion_3D_post
    
    # combine liver and lesion mask
    mask_3D = mask_liver.copy()
    mask_3D[np.where(mask_lesion==1)] = 2
    
    # save as nrrd file
    print("save as nrrd file")
    label_sitk, seg_sitk = save_mask_as_nrrd(
        mask_3D,
        num_classes=3,
        image_sitk=image_sitk,
        newsize=oldsize,
        image_filepath=filepath,
        save_path=os.path.join( save_path,sample_name ),
        )

  0%|          | 0/5 [00:00<?, ?it/s]

>> 0000506023-a
Size before resampling:  (512, 512, 88)
Size after resampling:  (380, 380, 176)
shape before crop:  (380, 380, 176)
shape after crop:  (238, 300, 176)
getting mask_liver_3D


/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


post-process for predicted liver mask
getting mask_lesion_3D
post process lesion mask
save as nrrd file
Segment0_Extent 63 269 107 255 3 165
Segment1_Extent 95 216 120 246 35 144
Size after resampling:  (512, 512, 88)


 20%|██        | 1/5 [00:29<01:57, 29.37s/it]

...

Segment0_Extent 63 246 37 181 57 173
Segment1_Extent 102 172 59 130 105 136
Size after resampling:  (456, 273, 92)
>> 10553469-a
Size before resampling:  (480, 322, 92)
Size after resampling:  (380, 255, 184)
shape before crop:  (255, 380, 184)
shape after crop:  (236, 329, 183)
getting mask_liver_3D
post-process for predicted liver mask
getting mask_lesion_3D
post process lesion mask
save as nrrd file
Segment0_Extent 55 258 37 213 17 182
Segment1_Extent 90 216 123 146 66 137
Size after resampling:  (480, 322, 92)


 80%|████████  | 4/5 [01:58<00:29, 29.50s/it]

>> 0000496938-a
Size before resampling:  (320, 200, 72)
Size after resampling:  (420, 262, 180)
shape before crop:  (262, 420, 180)
shape after crop:  (228, 309, 179)
getting mask_liver_3D
post-process for predicted liver mask
getting mask_lesion_3D
post process lesion mask
save as nrrd file


100%|██████████| 5/5 [02:25<00:00, 29.20s/it]

Segment0_Extent 74 285 45 205 24 167
Segment1_Extent 110 137 74 177 101 151
Size after resampling:  (320, 200, 72)
